# Human Development

We're going to extract from a UK Office for National Statistics xlx spreadsheet on human development.

This id a particularly oddly structured table in terms of machine readibility and serves as probably our most complex example.

tidychecf can handle this just fine but _remember this is an edge case_ example so **do all of the other examples first** and maybe even work through some of your own transforms before tackling this one.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/human-development-2008.xlsx", tables="Table 3")
preview(table, bounded="A1:W32")

From an xlsx source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/human-development-2008.xlsx).

## Requirements

Given this a particularly bizarely shaped dataset, we're going to do a bit of fair bit of preamble here to explain this data structure before we get into the extraction logic.

The following is a labelled preview to help explain the scenario.

In [ ]:
from IPython.display import IFrame
IFrame(src='./preamble-preview.html', width='100%', height=2500)

## Requirements

- We'll take the infroamtion alone the top as `Metric`
- We'll take `Country` as per the preview above
- We're gonna want `Category` (HUMAN DEVELOPMENT etc) as the non numeric values in column A, discounting headers at the top of the page.
- We'll want `Data Marker` as whatevers to the right of an observation, blank is fine - this **does not apply** to observations from column A but we will need to guarantee a consistent structure.
- We'll need a unit of measure to differentiate between number which are ranks vs numbers which as percentages
- We'll take observations as per the above, the the header column in the middle of them (and the way data markers are being used) necessitates us making two separate extractions (which we'd then combine) as follows:
    - extraction 1: observationsto the **left** of country, constant blank data markers
    - extractoin 2: observations to the **right** of country, extracted data markers.


In [ ]:
from tidychef import acquire, preview
from tidychef.direction import right, down, left
from tidychef.output import TidyData, Column
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/human-development-2008.xlsx", tables="Table 3")

# So we're going to need slightly different instruction for observations in column A vs all the other observations, so we're gonna
# create a jobs variable to track this
jobs = [
    {
        "direction to obs": left,
        "get obs": table.cell_containing_string("HDI rank").fill(down).is_not_blank().is_numeric().label_as("Observations"),
        "get data markers": None
    },
    {
        "direction to obs": right,
        "get obs": table.excel_ref("B7:B14").expand(right).is_not_blank().fill(down).is_not_blank().label_as("Observations"),
        "get data markers": table.excel_ref("B7:B14").expand(right).is_not_blank().fill(down).is_not_blank().shift(right).label_as("Data Marker")
    }
]

# Given the exeptionally messy header structure we'll use a lookup to didy things up.
# The key is what extracted - the value what it'll be post extraction.
lookup = {
    '$1 a day \n1990-2005d':  "Population below income poverty line (%), $1 a day, 1990-2005",
     '$2 a day \n1990-2005d ': "Population below income poverty line (%), $2 a day, 1990-2005",
     'Adult illiteracy rateb,†\n(% aged 15 and older)\n1995-2005 ': "Adult illiteracy rate (% aged 15 and older) 1995-2005",
     'Children under weight for age†\n(% under age 5)\n1996-2005d ': "Children under weight for age (% under age 5), 1996-2005",
     'HDI rank': "HDI rank",
     'HPI-1 rank minus income poverty rankc': "Human povery index rank minus income poverty rank",
     'National poverty line           1990-2004d': "National poverty line, 1990-2004",
     'Population not using an improved water source†\n(%)\n2004': "Population not using an improved water source, 2004",
     'Probability at birth of not surviving to age 40a,†\n(% of cohort)\n2000-05' :"Probability at birth of not surviving to age 40, % of cohort 2000-5",
     'Rank': "Human poverty index - Rank",
     'Value \n (%)': "Human poverty index - Value(%)"
}

all_tidy_data = []

for job in jobs:
    
    # We don't want anyting below NOTES in column A
    unwanted = table.column('A').cell_containing_string("NOTES").expand_to_box()
    
    header = table.excel_ref("9:13").is_not_blank().label_as("Headers")
    
    # Category is the non observation data in in column A
    category = table.column("A").cell_containing_string("HIGH HUMAN DEVELOPMENT").expand(down).is_not_blank().is_not_numeric().label_as("Category") - unwanted
    
    # Get the observations via whatever mechanisms is appropriate for this job
    observations = job["get obs"] - unwanted
    
    # Get countries from column B
    country = table.column("B").is_not_blank().label_as("Country") - unwanted
    
    # We only get data markers in some sceanrios
    if job["get data markers"] is not None:
        data_markers = job["get data markers"] - unwanted
        preview(observations, header, category, country, data_markers, bounded="A1:Q48")
    else:
        preview(observations, header, category, country, bounded="A1:Q48")
        
    # The tidy data definition both jobs have in common
    # Note: we're not using a .get() with the lookup, this is deliberate, if you must
    # use a lookup o this kind (dont unless you have to) always be explicity, its better
    # it raises a KeyError than fails silently.
    tidy_data = TidyData(
        observations,
        Column(header.attach_directly(down), apply=lambda x: lookup[x]), # use the lookup to tidy this up, see nuances
        Column(category.attach_closest(down)),
        Column(country.attach_directly(job["direction to obs"]))
    )
    
    # Where we have a data_marker selection we use it, where we do not we
    # include a blank Data Marker column to make sure both jobs result in
    # an identical structure.
    # There's an important thing here. The TidyData class uses lazy evauation,
    # i.e the transformation does not happen until it needs to - this is what
    # allows us to add columns post instantiation.
    if job["get data markers"]:
        tidy_data.add_column(Column(data_markers.attach_directly(left)))
    else:
        tidy_data.add_column(Column.constant("Data Marker", ""))
    
    all_tidy_data.append(tidy_data)
    
                                 
# Now we join them
tidy_data = TidyData.from_tidy_list(all_tidy_data)

## Nuances

### Using the lookup dictionary

In general programming (and in python in particular) there's a **KISS** (Keep it simple stupid) rule, which applies equally to your extractions.

We _could_ have written a lot of clever code to get those headers, but honetly, you'll often find a simple lookup dictionary is just easier to maintain on blance, at least with a structure with this level of convolution - a dictionary lookup is a very light touch thing to do when paired with apply.

But **one piece of advice**, you'dll notice I'm doing `lookup[x]` not `lookup.get(x, x)` because (in the hyperthetical scenario of running this against future versions of the data source) I'd _want_ a `KeyError` if the header text changes.

That won't always be the case (maybe you only want to change a _specific_header value and leave all the others as-is). So if you're hazy on the python trade offs of `dictionary[key]` vs `dictionary.get(key, key)` it's probably worth doing a little reading.

# Outputs

The tidy data can be [downloaded here](./human-development.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [ ]:
print(all_tidy_data)